# Linear Transformations
---
**Name:** Jonh Alexis Buot <br>
**Date:** December 2023 <br>
**Course:** CS3101N <br>
**Task:** Assignment - Linear Transformations

---

In [1]:
import random
import numpy as np

# Code Challenges

**1. Develop a python function from scratch that will find the determinants of any $n$ x $n$ matrix.**

In [2]:
def mat_det(A):
# =============================================================================
#   Calculate the determinant of a square matrix A.
# =============================================================================
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."
    
    n = len(A)
    if   n == 0: return 1.
    elif n == 1: return A[0][0]
    elif n == 2: return A[0][0] * A[1][1] - A[0][1] * A[1][0] 

    det = 0.
    for col, cofactor in enumerate(A[0]):
        minor = [[A[i][j] for j in range(n) if j != col] for i in range(1, n)]
        det += (-1) ** col * cofactor * mat_det(minor)

    return det

**2. Develop a python function from scratch that will find both the eigenvectors and eigenvalues of any $n$ x $n$ matrix.**

In [3]:
def mat_eig(A, max_iterations = 50000):
# =============================================================================
#   Calculate the eigenvectors and eigenvalues of a square matrix A.
#   
#   Note: This uses power method with Weilandt deflation.
#   https://services.math.duke.edu/~jtwong/math361-2019/lectures/Lec10eigenvalues.pdf
# =============================================================================
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."

    n = len(A) 
    if n == 0: return [], []
    if n == 1: return [A[0][0]], [[1.]]

    eigenvectors = []
    eigenvalues = []

    for _ in range(n):
        # Generate random guess for eigenvector with ||eigvec|| = 1
        eigvec = [random.random() for _ in range(n)]
        norm_eigvec = sum(a ** 2 for a in eigvec) ** 0.5
        eigvec = [a / norm_eigvec for a in eigvec]

        # Power Method
        for _ in range(max_iterations):
            tf_eigvec = [sum(a * b for a, b in zip(row, eigvec)) for row in A]
            
            norm_tf = sum(a ** 2 for a in tf_eigvec) ** 0.5
            eigvec = [a / norm_tf for a in tf_eigvec]
            
        eigval = sum(a * b for a, b in zip(tf_eigvec, eigvec))
        eigenvalues.append(eigval)
        eigenvectors.append(eigvec)

        # Deflate Matrix with Weilandt deflation
        eigenval_outer_product = [[a * b for b in eigvec] for a in tf_eigvec]
        A = [[A[i][j] - eigenval_outer_product[i][j] for j in range(n)] for i in range(n)]

    return eigenvalues, [list(row) for row in zip(*eigenvectors)]

**3. Test your functions from a randomly generated $n$ x $n$ matrix.**

- Testing and comparing my determinant implementation to the `numpy` implementation for an $n$ x $n$ matrix.

In [4]:
A = np.random.random((8, 8)).astype("float64") * 5

my_result = mat_det(A.tolist())
np_result = np.linalg.det(A)

print("-- A --", A, sep="\n")
print("\n-- My det(A) --", my_result, sep="\n")
print("\n-- Numpy det(A) --", np_result, sep="\n")
print("\nAre my results equal to numpy?", "Yes." if np.isclose(my_result, np_result) else "No.")

-- A --
[[2.61654799 1.92755718 0.80934912 2.07062851 4.38792179 2.08568381
  3.07140397 2.97398685]
 [1.57326172 3.28994004 1.1426231  2.75226726 2.77261071 2.61869971
  4.04993022 1.62266107]
 [2.99787751 3.75953669 0.40446025 0.53956571 3.58952259 4.99530672
  4.79429782 1.60752208]
 [1.99797279 0.66882465 2.12216409 1.06980442 0.14800765 3.74073806
  1.53735935 2.29732435]
 [0.29288092 0.9396201  2.95297398 1.37040809 0.31911604 1.70463093
  1.27872404 2.65460038]
 [4.14582085 3.60046556 4.57750194 3.50063362 2.25634093 1.30662281
  1.55736113 0.45325291]
 [2.85754194 4.94858426 3.59183662 4.75397572 4.4283282  4.87825044
  4.42220078 2.01641828]
 [2.86462575 3.72657255 2.2160887  1.30797059 1.25828481 2.50840064
  0.64466336 1.84662126]]

-- My det(A) --
6805.065697917604

-- Numpy det(A) --
6805.065697917598

Are my results equal to numpy? Yes.


- Testing and comparing my eigenvalue and eigenvector algorithm implementation to the `numpy` implementation for an $n$ x $n$ matrix.

In [19]:
A = np.random.random((3, 3)).astype("float64") * 5

np_ev, np_evl = np.linalg.eig(A)
m_ev, m_evl = mat_eig(A.tolist())
m_ev, m_evl = np.array(m_ev), np.array(m_evl)

print("-- A --", A, sep="\n")
print("\n-- My Eigenvalues and Eigenvectors --", m_ev, m_evl, sep="\n")
print("\n-- Numpy Eigenvalues and Eigenvectors --", np_ev, np_evl, sep="\n")

print("\n\n", "Are the differences of eigenvalues close?", "Yes." if np.allclose(m_ev, np_ev) else "No.")

cosine_similarity = np.array([abs(npe.dot(me) / (np.linalg.norm(me) * np.linalg.norm(npe))) for npe, me in zip(np_evl.T, m_evl.T)]).T
print("\n-- Cosine Similarity of Eigenvalues --", cosine_similarity, sep="\n")

-- A --
[[4.95278041 0.9504671  2.18684775]
 [0.58380958 3.50930189 1.51983768]
 [4.17113516 2.09361879 4.54084863]]

-- My Eigenvalues and Eigenvectors --
[8.43705126 3.1700862  1.39579347]
[[ 0.56455005  0.44016942  0.93755896]
 [ 0.30360975 -0.89737194  0.22158244]
 [ 0.76753134  0.03121654  0.26811269]]

-- Numpy Eigenvalues and Eigenvectors --
[8.43705126 3.1700862  1.39579347]
[[-0.56455005 -0.44016358 -0.36536384]
 [-0.30360975  0.89737509 -0.47483098]
 [-0.76753134 -0.03120859  0.80065274]]


 Are the differences of eigenvalues close? Yes.

-- Cosine Similarity of Eigenvalues --
[1.        1.        0.2330992]
